In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
data=pd.read_csv('/content/Corona_NLP_train.csv',encoding='latin1')

In [ ]:
df=pd.DataFrame(data)

In [ ]:
df.head(20)

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive
3,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,Positive
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",Extremely Negative
5,3804,48756,"ÃT: 36.319708,-82.363649",16-03-2020,As news of the regionÂs first confirmed COVID...,Positive
6,3805,48757,"35.926541,-78.753267",16-03-2020,Cashier at grocery store was sharing his insig...,Positive
7,3806,48758,Austria,16-03-2020,Was at the supermarket today. Didn't buy toile...,Neutral
8,3807,48759,"Atlanta, GA USA",16-03-2020,Due to COVID-19 our retail store and classroom...,Positive
9,3808,48760,"BHAVNAGAR,GUJRAT",16-03-2020,"For corona prevention,we should stop to buy th...",Negative


In [ ]:
df.dtypes

UserName          int64
ScreenName        int64
Location         object
TweetAt          object
OriginalTweet    object
Sentiment        object
dtype: object

In [ ]:
df.isna().sum()

UserName            0
ScreenName          0
Location         8590
TweetAt             0
OriginalTweet       0
Sentiment           0
dtype: int64

In [ ]:
!pip install tweet-preprocessor

In [ ]:
import preprocessor as p
j=""
for i in df.OriginalTweet:
  p.set_options(p.OPT.URL, p.OPT.EMOJI,p.OPT.MENTION,p.OPT.HASHTAG,p.OPT.RESERVED,p.OPT.SMILEY,p.OPT.NUMBER)
  j=p.clean(i)
  df.OriginalTweet[i].replace(i,j)

In [ ]:
import re
a = re.compile("(@[A-Za-z0-9]+)|(#[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)")
tweet = []

for i in df["OriginalTweet"]:
  tweet.append(a.sub(" ", i))

df = pd.concat([df, pd.DataFrame(tweet, columns=["Preprocessed"])], axis=1, sort=False)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))     
vectoriser = TfidfVectorizer(stop_words=None)    

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
X_train = vectoriser.fit_transform(df["Preprocessed"])
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
y_train = encoder.fit_transform(df['Sentiment'])
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()
classifier.fit(X_train, y_train)
test_data = pd.read_csv("/content/Corona_NLP_test.csv",encoding='latin1')
test_df = pd.DataFrame(test_data)
a = re.compile("(@[A-Za-z0-9]+)|(#[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)")
tweet = []
for i in test_df["OriginalTweet"]:
  tweet.append(a.sub(" ", i))
test_df = pd.concat([test_df, pd.DataFrame(tweet, columns=["Preprocessed"])], axis=1, sort=False)

In [ ]:
X_test = vectoriser.transform(test_df["Preprocessed"])
y_test = encoder.transform(test_df["Sentiment"])
y_pred = classifier.predict(X_test)
pred_df = pd.DataFrame({'Actual': y_test, 'Prediction': y_pred})
pred_df.head()

,Actual,Prediction
0,0,2
1,4,4
2,1,4
3,2,2
4,3,2
